<a href="https://colab.research.google.com/github/davidmonterocrespo24/test_ai/blob/main/example_colab_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para entrenar un modelo de lenguaje GPT-2 en Python usando el código de una carpeta, necesitarás seguir los siguientes pasos:

Instalar las dependencias necesarias:

In [ ]:
!pip install tensorflow==2.3.0
!pip install tensorflow_datasets==3.2.1
!pip install transformers==3.5.1

Importar las bibliotecas necesarias:


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import os


Cargar los datos:

In [ ]:
path = 'modelo'
text = ''
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as f:
        text += f.read()


Tokenizar los datos:

In [ ]:
# Cargar el modelo y el tokenizador
model = TFGPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Ajustar la longitud máxima de secuencia
model.config.max_position_embeddings = 102048
tokenizer.model_max_length = 102048
tokenized_text = tokenizer.encode(text)

Crear los conjuntos de entrenamiento y prueba:

In [ ]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(tokenized_text)-block_size+1, block_size):
    examples.append(tokenized_text[i:i+block_size])
inputs, labels = [], []
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


Definir el modelo:


In [ ]:
model = TFGPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)


Compilar el modelo:

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), loss=[loss_fn, *[None] * model.config.n_layer])


Entrenar el modelo:

In [ ]:
EPOCHS = 1
model.fit(dataset, epochs=EPOCHS)

Guardar el modelo:

In [ ]:
model.save_pretrained('modelo')

Aquí te muestro un ejemplo de cómo utilizar el modelo entrenado anteriormente para generar texto:

In [ ]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

# Cargar el modelo y el tokenizador
model = TFGPT2LMHeadModel.from_pretrained('modelo')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Generar texto
prompt = "En un lugar de la Mancha"
input_ids = tokenizer.encode(prompt, return_tensors='tf')
output = model.generate(input_ids, max_length=100, do_sample=True, top_k=50)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)
